In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
from oatlib import sensor, method, oat_utils # https://gitlab.com/freewat/freewat/tree/develop/oat
import datetime
from dateutil import parser
from datetime import timedelta
import scipy
import numpy as np
from pandas import concat
import pandas as pd
import pylab as plot
import numpy as np

In [ ]:
sensors_list = [
    {
        'name': '4onse',
        'service': 'https://geoservice.ist.supsi.ch/4onse/sos',
        'basic_auth': ('xx', 'xx'),
        'observed_properties': [
            {
                'prop': 'temperature',
                'csv': './output/PCB_temperature_orig.csv',
                'unit': 'Celsius',
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:temperature'
                ),
                'procedure': 'TREVANO_PCB',
                'labels': {
                    'name': 'Temperature',
                    'unit': '$^\circ$C',
                    'x_limit_margin': 5,
                    'y_limit_margin': 5,
                    'limit': 1
                },
                'method': 'standard'
            },
            {
                'prop': 'humidity',
                'csv': './output/PCB_humidity_orig.csv',
                'unit': '%',
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:humidity'
                ),
                'procedure': 'TREVANO_PCB',
                'labels': {
                    'name': 'Humidity',
                    'unit': '$^\circ$C',
                    'x_limit_margin': 5,
                    'y_limit_margin': 5,
                    'limit': 4
                },
                'method': 'standard'
            },
            {
                'prop': 'pressure',
                'csv': './output/PCB_Pressure_orig.csv',
                'unit': 'hPa',
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:pressure'
                ),
                'procedure': 'TREVANO_PCB',
                'labels': {
                    'name': 'Pressure',
                    'unit': 'hPa',
                    'x_limit_margin': 3,
                    'y_limit_margin': 3,
                    'limit': 1
                },
                'method': 'standard'
            }
        ]
    },
    {
        'name': 'SUPSI',
        'service': 'xx',
        'basic_auth': ('xx', 'xx'),
        'observed_properties': [
            {
                'prop': 'temperature',
                'csv': './output/SUPSI_temperature_orig.csv',
                'unit': 'Celsius',
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:temperature'
                ),
                'procedure': 'T_TRE',
                'labels': {
                    'name': 'Temperature',
                    'unit': '$^\circ$C',
                    'x_limit_margin': 5,
                    'y_limit_margin': 5,
                    'limit': 1
                },
                'method': 'standard'
            },
            {
                'prop': 'humidity',
                'csv': './output/SUPSI_humidity_orig.csv',
                'unit': '%',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:humidity'
                ),
                'procedure': 'H_TRE',
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'labels': {
                    'name': 'Humidity',
                    'unit': '$^\circ$C',
                    'x_limit_margin': 5,
                    'y_limit_margin': 5,
                    'limit': 4
                },
                'method': 'standard'
            },
            {
                'prop': 'Pressure',
                'csv': './output/SUPSI_Pressure_orig.csv',
                'unit': 'hPa',
                'observed_property': (
                    'urn:ogc:def:parameter:x-istsos:1.0:meteo:air:pressure'
                ),
                'aggregate_function': 'AVG',
                'aggregate_interval': 'PT10M',
                'procedure': 'B_TRE',
                'labels': {
                    'name': 'Pressure',
                    'unit': 'hPa',
                    'x_limit_margin': 3,
                    'y_limit_margin': 3,
                    'limit': 1
                },
                'method': 'standard'
            }
        ]
    }
]

In [ ]:
class CompareStation():
    def __init__(self, sensors_list, event_time):
        self.event_time = event_time
        self.sensors_list = sensors_list
        self.interval = timedelta(days=15)
        self.legend_params = {
            'legend.fontsize': 5,
            'legend.handlelength': 2
        }
        
    def check_compatibility(self):
        if len(self.sensors_list)==2:
            a_sensor_obs = self.sensors_list[0]['observed_properties']
            b_sensor_obs = self.sensors_list[1]['observed_properties']
            if len(a_sensor_obs) == len(b_sensor_obs):
                for a_ob in a_sensor_obs:
                    res = list(
                        filter(
                            lambda x:  x['observed_property'] == a_ob['observed_property'], b_sensor_obs
                        )
                    )
                    if res[0]['unit'] == a_ob['unit']:
                        if res[0]['aggregate_function'] == a_ob['aggregate_function']:
                            if res[0]['aggregate_interval'] == a_ob['aggregate_interval']:
                                return True
                            else:
                                raise Exception(
                                    'The aggregate interval {} does not match the one chosen for the observed property {} in station {}.'.format(
                                        a_ob['aggregate_interval'], res['observed_property'][0], self.sensors_list[1]['name']
                                    )
                                )
                        else:
                            raise Exception(
                                'The aggregate function {} does not match the one chosen for the observed property {} in station {}.'.format(
                                    a_ob['aggregate_function'], res['observed_property'][0], self.sensors_list[1]['name']
                                )
                            )
                    else:
                        raise Exception(
                            'The unit {} used for the procedure {} does not exist in station {}.'.format(
                                a_ob['unit'], a_ob['observed_property'][0], self.sensors_list[1]['name']
                            )
                        )
                    
            else:
                raise Exception('The observed properties number of the sensors dismatches.')
        else:
            raise Exception('The comparison analysis needs two sensors.')
            
    # da istSOS
    def fetch_data(self):
        for sensor_i in self.sensors_list:
            column_names = []
            sensor_stats = None
            id_obp = 0
            for observed_property in sensor_i['observed_properties']:
                print(
                    'Event time --> {}'.format(self.event_time)
                )
                print(
                    'Start fetching procedure: {}\nObserved property: {}\n'.format(
                        sensor_i['name'], observed_property['prop']
                    )
                )
                column_names_in = []
                begin_end_time = self.event_time.split('/')
                begin_time = parser.parse(begin_end_time[0])
                end_time = parser.parse(begin_end_time[1])
                next_start = begin_time
                
                while end_time >= next_start:
                    if (next_start == end_time):
                        break
                    else:
                        stop_time = next_start + self.interval
                        event_time = '{}/{}'.format(
                            next_start.isoformat(), stop_time.isoformat()
                        )
                        print(
                            sensor_i['service'],
                            observed_property['observed_property'],
                            observed_property['procedure'],
                            sensor_i['basic_auth'],
                            observed_property['aggregate_function'],
                            observed_property['aggregate_interval'],
                            event_time
                        )

                        SENSOR_TMP = sensor.Sensor(
                            name=sensor_i['name'],
                            prop=observed_property['prop'],
                            unit=observed_property['unit']
                        )
                        SENSOR_TMP.ts_from_istsos(
                            service=sensor_i['service'],
                            observed_property=observed_property['observed_property'],
                            procedure=observed_property['procedure'],
                            basic_auth=sensor_i['basic_auth'],
                            aggregate_function=observed_property['aggregate_function'],
                            aggregate_interval=observed_property['aggregate_interval'],
                            event_time=event_time
                        )
                    if next_start == begin_time:
                        SENSOR = SENSOR_TMP.copy()
                    else:
                        SENSOR.ts = concat([SENSOR.ts, SENSOR_TMP.ts], verify_integrity=True)
                    if (next_start + self.interval) > end_time:
                        next_start = end_time
                    else:
                        next_start += + self.interval
                sensor_i['observed_properties'][id_obp]['df'] = SENSOR
                MAX_COL = observed_property['prop'][0].upper()+'MAX'
                TIME_MAX_COL = 'TIME_' + MAX_COL
                MIN_COL = observed_property['prop'][0].upper()+'MIN'
                TIME_MIN_COL = 'TIME_' + MIN_COL
                MEAN_COL = observed_property['prop'][0].upper()+'MEAN'
                COUNT = '{}_COUNT'.format(MEAN_COL)

                daily_max = oat_utils.sensorStats(
                    SENSOR, stat='max',
                    column_name=MAX_COL
                )
                daily_min = oat_utils.sensorStats(
                    SENSOR, stat='min',
                    column_name=MIN_COL
                )
                daily_mean = oat_utils.sensorStats(
                    SENSOR, stat='mean',
                    column_name=MEAN_COL
                )

                column_names = column_names + [
                    MAX_COL, TIME_MAX_COL, MIN_COL, TIME_MIN_COL, MEAN_COL, COUNT
                ]

                daily_stats = SENSOR.copy()

                daily_stats.ts = daily_min.ts[[MIN_COL]].join(
                    daily_min.ts[[TIME_MIN_COL]]
                ).join(
                    daily_max.ts[[MAX_COL]]
                ).join(
                    daily_max.ts[[TIME_MAX_COL]]
                ).join(
                    daily_mean.ts[[MEAN_COL]]
                ).join(
                    daily_mean.ts[[COUNT]]
                )

                if sensor_stats:
                    for c in daily_stats.ts.columns.values:
                        sensor_stats.ts = sensor_stats.ts.join(daily_stats.ts[c])
                else:
                    sensor_stats = daily_stats.copy()
                id_obp+=1
            sensor_i['stats'] = sensor_stats
            sensor_stats.save_to_csv(
                './output/' + sensor_i['name'] + '_daily_stats.csv', columns=column_names
            )
        print('#################\nFetching ended.')
        
    def filter_stats(self, count, quality_index=None):
        # data filtered on the count of data used for the mean calculation.

        for sensor_i in self.sensors_list:
            columns_name = sensor_i['stats'].ts.columns.values.tolist()
            count_columns = list(filter(lambda x: 'COUNT' in x, columns_name))
            sensor_i['stats_filtered'] = sensor_i['stats'].ts[sensor_i['stats'].ts[count_columns[0]]>=count]
#                 dfs_filtered_1.append(sensor.ts[sensor.ts['HMEAN_COUNT']>=143])

    def sync_stats(self):
        
        idx1 = pd.Index(self.sensors_list[0]['stats_filtered'].index.values)
        idx2 = pd.Index(self.sensors_list[1]['stats_filtered'].index.values)
        diff1 = idx2.difference(idx1)
        diff2 = idx1.difference(idx2)
        self.sensors_list[0]['stats_filtered_synced'] = self.sensors_list[0]['stats_filtered'].drop(diff2)
        self.sensors_list[1]['stats_filtered_synced'] = self.sensors_list[1]['stats_filtered'].drop(diff1)
        
    def sync_dfs(self):
        
        """
        TODO --> Find a better way to sync DF
        """
        
        idx2_diff = []
        idx1_diff = []

        sensors_orig_list2 = []

        for sensor_ in self.sensors_list:
            for i in range(len(sensor_['observed_properties'])):
                toat = sensor_['observed_properties'][i]
                toat['df'].ts = sensor_['observed_properties'][i]['df'].ts.dropna(how='any')
                sensor_['observed_properties'][i]['df_synced'] = toat['df']
                sensors_orig_list2.append(toat)

        for sensor_ in sensors_orig_list2:
            idx1 = pd.Index(sensor_['df'].ts.index.values)
            for i in sensors_orig_list2:
                idx2 = pd.Index(i['df'].ts.index.values)
                diff = idx2.difference(idx1)
                if diff.size > 0:
                    idx2_diff.append(diff)
                diff = idx1.difference(idx2)
                if diff.size > 0:
                    idx1_diff.append(diff)


        for sensor_ in self.sensors_list:
            for i in range(len(sensor_['observed_properties'])):
                toat = sensor_['observed_properties'][i]['df_synced'].copy()
                for diff in idx1_diff:
                    try:
                        toat.ts = sensor_['observed_properties'][i]['df_synced'].ts.drop(diff)
                    except Exception as e:
                        continue
                for diff in idx2_diff:
                    try:
                        toat.ts = toat.ts.drop(diff)
                    except Exception as e:
                        continue
                sensor_['observed_properties'][i]['df_synced'] = toat

    def plot_orig_values(self):
        letters = ['a','b','c','d','e','f']
        
        for obp in self.sensors_list[0]['observed_properties']:
            obp_ref = list(
                filter(lambda x: x['observed_property'] == obp['observed_property'], self.sensors_list[1]['observed_properties'])
            )[0]
            obp_unit = obp['labels']['unit']
            
            label_ = '{}_{}'.format(obp['prop'][0].upper(), obp['df'].name)
            label_ref = '{}_{}'.format(obp_ref['prop'][0].upper(), obp_ref['df'].name)
            obp['df'].ts['data'].plot(legend=True, label=label_)
            obp_ref['df'].ts['data'].plot(legend=True, label=label_ref)


            plt.ylabel('{} ({})'.format(obp['labels']['name'], obp['labels']['unit']))
            plt.xlabel('Date')
            plt.legend(loc='best', fontsize='medium')
            plt.savefig('{}_full.png'.format(obp['prop']))
            plt.close()
            A_matrix = obp['df_synced'].ts['data'].values
            B_matrix = obp_ref['df_synced'].ts['data'].values
            matrix =  np.concatenate(
                (
                    obp['df'].ts.dropna()['data'].as_matrix(),
                    obp_ref['df'].ts.dropna()['data'].as_matrix()
                ), axis=0
            )
            print(
                '%s R-squared (Pearson): %s \n%s R-squared (Kendall): %s \n%s R-squared (Speaman): %s' % (
                    obp['labels']['name'], obp['df_synced'].ts['data'].corr(obp_ref['df_synced'].ts['data']),
                    obp['labels']['name'], obp['df_synced'].ts['data'].corr(obp_ref['df_synced'].ts['data'], method='kendall'),
                    obp['labels']['name'], obp['df_synced'].ts['data'].corr(obp_ref['df_synced'].ts['data'], method='spearman')
                )
            )
            # scatter plot
            plt.title('{})'.format(letters[0]), loc='left')
            percentiles = [np.percentile(matrix, 5), np.percentile(matrix, 95)]
            residuals = obp['df_synced'].ts - obp_ref['df_synced'].ts
            quantiles = [
                residuals.dropna().quantile(.05)['data'],
                residuals.dropna().quantile(.95)['data']
            ]
            min_1 = obp['df_synced'].ts['data'].min()
            min_2 = obp_ref['df_synced'].ts['data'].min()
            max_1 = obp['df_synced'].ts['data'].max()
            max_2 = obp_ref['df_synced'].ts['data'].max()
            if min_1 < min_2:
                x_limit = min_1 - 5
            else:
                x_limit = min_2 - 5
            if max_1 < max_2:
                y_limit = max_2 + 5
            else:
                y_limit = max_1 + 5

            limit_num = obp['labels']['limit']
            limit_label = '{} {}'.format(limit_num, obp_unit)

            plt.plot(
                [
                    x_limit,
                    y_limit
                ],[
                    x_limit,
                    y_limit
                ], c='black', linewidth=1
            )
            plt.plot(
                [
                    obp['df_synced'].ts.dropna()['data'].min(),
                    obp['df_synced'].ts.dropna()['data'].max()
                ],[
                    obp['df_synced'].ts.dropna()['data'].min()+limit_num,
                    obp['df_synced'].ts.dropna()['data'].max()+limit_num
                ], c='green', linestyle='dashed', linewidth=1, label='-%s limit' % (limit_label)
            )

            plt.plot(
                [
                    obp['df_synced'].ts.dropna()['data'].min(),
                    obp['df_synced'].ts.dropna()['data'].max()
                ],[
                    obp['df_synced'].ts.dropna()['data'].min()-limit_num,
                    obp['df_synced'].ts.dropna()['data'].max()-limit_num
                ], c='red', linestyle='dashed', linewidth=1, label='+%s limit' % (limit_label)
            )
            scatter_plot = plt.scatter(A_matrix, B_matrix, marker='+')
            axes = plt.gca()
            axes.set_xlim([x_limit,y_limit])
            axes.set_ylim([x_limit,y_limit])


            plt.xlabel('{} {} ({})'.format(obp['labels']['name'], obp['labels']['unit'], obp['df_synced'].name))
            plt.ylabel('{} {} ({})'.format(obp['labels']['name'], obp['labels']['unit'], obp_ref['df_synced'].name))
            plt.legend(loc='best', fontsize='medium')
            plt.savefig('{}_scatter.png'.format(obp['prop']))
            plt.close()
            
            # density plot
            min_ = residuals['data'].min()
            max_ = residuals['data'].max()
            x_limit_sx = min_ - limit_num
            x_limit_dx = max_ + limit_num
            plt.title('{})'.format(letters[0]), loc='left')
            axes = plt.gca()
            axes.set_xlim([x_limit_sx,x_limit_dx])
            res_plt_den = residuals['data'].plot.density()
            plt.xlabel('{} {} ({})'.format(obp['labels']['name'], obp['labels']['unit'], obp['df_synced'].name))
            mean_val = residuals['data'].mean()
            std_val = residuals['data'].std()
            print('Mean abs error: %s' % (mean_val))
            print('Std error: %s' % (std_val))
            print('First quantile, second quantile: {}, {}'.format(quantiles[0], quantiles[1]))
            plt.axvline(mean_val, c= 'black', linestyle='dashed', linewidth=1, label='Mean')
            res_plt_den.axvspan(quantiles[0], quantiles[1], alpha=0.3, color='red', label="2$\sigma$")
            plt.legend(loc='best', fontsize='medium')
            plt.savefig('{}_density.png'.format(obp['prop']))
            plt.close()
            

    def plot_stats_values_residuals(self):
        
        print('Plotting')
        plot.rcParams.update(self.legend_params)
        letters = ['a','b','c','d','e','f']
        
        columns = self.sensors_list[0]['stats_filtered_synced'].columns.values

        s_1 = sensors_list[0]['name']
        s_2 = sensors_list[1]['name']
        first_sensor = sensors_list[0]['stats_filtered_synced']

        second_sensor = sensors_list[1]['stats_filtered_synced']
        residuals = first_sensor - second_sensor
        id_col = 0
        for col in columns:
            if 'COUNT' not in col:
                col_ = '{}_{}'.format(col, s_1)
                col_ref = '{}_{}'.format(col, s_2)
                col_res = '{}_RESIDUALS'.format(col)
                file_name = '{}_res.png'.format(col)
                scatter_name = '{}_scatter_plot.png'.format(col)
                residual_name = '{}_residual_plot.png'.format(col)
                density_name = '{}_density_plot.png'.format(col)
                id_obp = int(id_col/6)
                obp_name = sensors_list[0]['observed_properties'][id_obp]['labels']['name']
                obp_unit = sensors_list[0]['observed_properties'][id_obp]['labels']['unit']
                sensors_list[0]['observed_properties'][id_obp]['labels']['name']
                scatter_x_label = '{} {} ({})'.format(obp_name, obp_unit, s_1)
                scatter_y_label = '{} {} ({})'.format(obp_name, obp_unit, s_2)
                scatter_y_residual_label = 'Residuals {} ({})'.format(col, obp_unit)
                gen_label = '{} {}'.format(obp_name, obp_unit)
                res_label = 'Residuals {}'.format(obp_unit)
                id_col+=1


                if 'TIME' not in col:
                    first_sensor[col].plot(legend=True, label=col_)
                    second_sensor[col].plot(legend=True, label=col_ref)
                    plt.ylabel(gen_label)
                    plt.xlabel('Date')
                    plt.legend(loc='best', fontsize='medium')
                    plt.savefig(file_name)
                    plt.close()
                    A_matrix = first_sensor[col].values
                    B_matrix = second_sensor[col].values

                    matrix =  np.concatenate(
                        (
                            first_sensor.dropna()[col].values,
                            second_sensor.dropna()[col].values
                        ), axis=0
                    )

                    print(
                        '%s R-squared (Pearson): %s \n%s R-squared (Kendall): %s \n%s R-squared (Speaman): %s' % (
                            col, first_sensor[col].corr(second_sensor[col]),
                            col, first_sensor[col].corr(second_sensor[col], method='kendall'),
                            col, first_sensor[col].corr(second_sensor[col], method='spearman')
                        )
                    )

                    # scatter plot
                    percentiles = [np.percentile(matrix, 5), np.percentile(matrix, 95)]
                    residuals = first_sensor - second_sensor
                    quantiles = [
                        residuals.dropna().quantile(.05)[col],
                        residuals.dropna().quantile(.95)[col]
                    ]
                    min_1 = first_sensor[col].min()
                    min_2 = second_sensor[col].min()
                    max_1 = first_sensor[col].max()
                    max_2 = second_sensor[col].max()
                    if min_1 < min_2:
                        x_limit = min_1 - sensors_list[0]['observed_properties'][id_obp]['labels']['x_limit_margin']
                    else:
                        x_limit = min_2 - sensors_list[0]['observed_properties'][id_obp]['labels']['x_limit_margin']
                    if max_1 < max_2:
                        y_limit = max_2 + sensors_list[0]['observed_properties'][id_obp]['labels']['y_limit_margin']
                    else:
                        y_limit = max_1 + sensors_list[0]['observed_properties'][id_obp]['labels']['y_limit_margin']

                    limit_num = sensors_list[0]['observed_properties'][id_obp]['labels']['limit']
                    limit_label = '{} {}'.format(limit_num, obp_unit)
                    if 'MIN' in col:
                        plt.title('{})'.format(letters[2]), loc='left')
                    elif 'MAX' in col:
                        plt.title('{})'.format(letters[1]), loc='left')
                    elif 'MEAN' in col:
                        plt.title('{})'.format(letters[3]), loc='left')
                    plt.plot(
                        [
                            x_limit,
                            y_limit
                        ],[
                            x_limit,
                            y_limit
                        ], c='black', linewidth=1
                    )
                    plt.plot(
                        [
                            first_sensor.dropna()[col].min(),
                            first_sensor.dropna()[col].max()
                        ],[
                            first_sensor.dropna()[col].min()+limit_num,
                            first_sensor.dropna()[col].max()+limit_num
                        ], c='green', linestyle='dashed', linewidth=1, label='-%s limit' % (limit_label)
                    )

                    plt.plot(
                        [
                            first_sensor.dropna()[col].min(),
                            first_sensor.dropna()[col].max()
                        ],[
                            first_sensor.dropna()[col].min()-limit_num,
                            first_sensor.dropna()[col].max()-limit_num
                        ], c='red', linestyle='dashed', linewidth=1, label='+%s limit' % (limit_label)
                    )
                    scatter_plot = plt.scatter(A_matrix, B_matrix, marker='+')
                    axes = plt.gca()
                    axes.set_xlim([x_limit,y_limit])
                    axes.set_ylim([x_limit,y_limit])

                    plt.xlabel(scatter_x_label)
                    plt.ylabel(scatter_y_label)
                    plt.legend(loc='best', fontsize='medium')
                    plt.savefig(scatter_name)
                    plt.close()

                    # density plot
                    res_plt_den = residuals[col].plot.density()
                    plt.xlabel(gen_label)
                    mean_val = residuals[col].mean()
                    std_val = residuals[col].std()
                    if 'MIN' in col:
                        plt.title('{})'.format(letters[2]), loc='left')
                    elif 'MAX' in col:
                        plt.title('{})'.format(letters[1]), loc='left')
                    elif 'MEAN' in col:
                        plt.title('{})'.format(letters[3]), loc='left')
                    print('Mean error: %s' % mean_val)
                    print('Std: %s' % std_val)
                    print('First quantile, second quantile: {}, {}'.format(quantiles[0], quantiles[1]))
                    plt.axvline(mean_val, c= 'black', linestyle='dashed', linewidth=1, label='Mean')
                    res_plt_den.axvspan(quantiles[0], quantiles[1], alpha=0.3, color='red', label="2$\sigma$")
                    plt.legend(loc='best', fontsize='medium')
                    plt.savefig(density_name)
                    plt.close()


In [ ]:
event_time = ('2018-07-01T00:00:00.000000Z/2018-02-28T00:00:00.000000Z')
cc = CompareStation(sensors_list, event_time)
if cc.check_compatibility():
    cc.fetch_data()

    cc.filter_stats(143)

    cc.sync_stats()

    cc.sync_dfs()

    cc.plot_orig_values()

    cc.plot_stats_values_residuals()
else:
    print('Procedures are not compatible')